<a href="https://colab.research.google.com/github/uadR1/nlphatespeech/blob/main/Hina/model_to_deplo_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/Colab Notebooks/Hate_Speech_NLP/nlphatespeech

/content/gdrive/MyDrive/Colab Notebooks/Hate_Speech_NLP/nlphatespeech


In [ ]:
!git pull

Already up to date.


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.3 MB/s 
     |████████████████████████████████| 7.6 MB 53.1 MB/s 
     |████████████████████████████████| 182 kB 77.8 MB/s 


In [ ]:
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, TFTrainer, TFTrainingArguments, DataCollatorWithPadding
import tensorflow as tf
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from matplotlib import pyplot as plt

# GPU 

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 25 13:20:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    32W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# READ DATA

In [ ]:
ls

'All Data'/   Duygu/      README.md
 Aras/        Himansha/   sentiment_analysis_with_whole_data.ipynb
 Daniel/      Hina/       Thilo/


In [ ]:

# Read Data 
df = pd.read_csv('Hina/df_all.csv')
df.drop(columns=df.columns[0], axis=1, inplace=True)
df.head()

,text,class
0,rt crooked hillary you didn't lose because yo...,explicit_hate
1,"rt estonian pm : "" i don't want blacks in es...",explicit_hate
2,are a bunch of religious bigots . i hope you e...,explicit_hate
3,hamas is the only one who lies and all those l...,explicit_hate
4,one of the questions never asked : if africans...,explicit_hate


In [ ]:

# drop NA 
df = df.dropna()

In [ ]:

# Check distribution of classes
df['class'].value_counts()

not_hate         35977
offensive        19190
implicit_hate     7100
explicit_hate     2519
Name: class, dtype: int64

## multiclass labelling of text

In [ ]:
# Combine classes for binary classification 

# Copy Dataframe first
df_multiclass = df.copy()

# Replace classes 
df_multiclass['class'] = np.where((df_multiclass['class'] == 'not_hate'), 0, df_multiclass['class'])
df_multiclass['class'] = np.where((df_multiclass['class'] == 'offensive'), 1, df_multiclass['class'])
df_multiclass['class'] = np.where((df_multiclass['class'] == 'implicit_hate'), 2, df_multiclass['class'])
df_multiclass['class'] = np.where((df_multiclass['class'] == 'explicit_hate'), 3, df_multiclass['class'])

# show head 
df_multiclass.head()

,text,class
0,rt crooked hillary you didn't lose because yo...,3
1,"rt estonian pm : "" i don't want blacks in es...",3
2,are a bunch of religious bigots . i hope you e...,3
3,hamas is the only one who lies and all those l...,3
4,one of the questions never asked : if africans...,3


In [ ]:
# Show Distribution
df_multiclass['class'].value_counts()

0    35977
1    19190
2     7100
3     2519
Name: class, dtype: int64

In [ ]:

# Get max number of words 
num_words = df_multiclass['text'].apply(lambda x:len(str(x).split()))
print('The comment with the most words consist of', num_words.max(),'words')

The comment with the most words consist of 268 words


# Train test Split

In [ ]:
# Create X  
X=list(df_multiclass['text']) 

# Create y 
y=list(df_multiclass['class']) 

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 123) 

In [ ]:
categories=sorted(list(set(y))) #set will return the unique different entries

In [ ]:
num_categories = len(categories)
num_categories

4

## Preprocessing

In [ ]:
# Set up an Object for RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

# Tokenize X Train & Test Set 
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors='tf')
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors='tf')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_input),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_input),
    y_test
))

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,   
    warmup_steps = 500,
    weight_decay=0.01,
    logging_steps=10,
    eval_steps = 10
)

In [ ]:
with training_args.strategy.scope():
    model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=num_categories)

trainer = TFTrainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
)

trainer.train()

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ResourceExhaustedError: ignored

In [ ]:
!git commit -m try


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@e72169e098b3.(none)')


In [ ]:
!git config --global user.email "tanvir.hina7@gmail.com"
!git config --global user.name "hinatanvir"

In [ ]:
%cd Hina

/content/gdrive/MyDrive/Colab Notebooks/Hate_Speech_NLP/nlphatespeech/Hina


In [ ]:
!git add model_to_deplo_roberta.ipynb

In [ ]:
!git commit -m try

[main 03d7bc8] try
 1 file changed, 1 insertion(+)
 create mode 100644 Hina/model_to_deplo_roberta.ipynb


In [32]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address
